#### 29.10.25, &copy; [Evhenii Kostin](https://github.com/DE123MasterProgram2025autumn/DE_Kostin), 2025

# Лабораторна робота №2: Інтеграція та об’єднання даних з різних джерел

__Мета:__ _освоїти повний цикл інтеграції реляційних даних у середовищі R: від завантаження даних з різноманітних джерел (CSV, Excel, Google Таблиці) до їх об’єднання за допомогою вертикальних (bind_rows) та горизонтальних (left_join, semi_join, anti_join) операцій. Навчитися будувати реалістичні ETL-конвеєри, які моделюють типові задачі інженера даних у production-середовищі._

### Варіант 2


In [10]:
import pandas as pd
import os

# --- 1. НАЛАШТУВАННЯ ТА СТВОРЕННЯ ПАПКИ ---
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Створено папку: {data_dir}")
else:
    print(f"Папка {data_dir} вже існує.")

# --- 2. ГЕНЕРАЦІЯ ДАНИХ (у вигляді Pandas DataFrame) ---

# Довідник студентів
students = pd.DataFrame({
  'student_id': ["S01", "S02", "S03", "S04", "S05", "S06"],
  'full_name': ["Іваненко Петро", "Коваленко Марія", "Сидоренко Валерій", "Мельник Олена", "Кравчук Андрій", "Лисенко Ірина"],
  'group': ["IT-21", "IT-21", "IT-22", "IT-22", "IT-21", "IT-22"],
  'faculty': ["FICT", "FICT", "FICT", "FICT", "FICT", "FICT"]
})

# --- Семестр 1 ---
grades_math_s1 = pd.DataFrame({'student_id': ["S01", "S02", "S03", "S04"], 'exam_score': [85, 92, 77, 60], 'project_score': [90, 88, 80, 65]})
grades_physics_s1 = pd.DataFrame({'student_id': ["S01", "S02", "S03", "S04"], 'exam_score': [95, 80, 82, 70], 'project_score': [90, 85, 80, 75]})
grades_cs_s1 = pd.DataFrame({'student_id': ["S01", "S02", "S03", "S04"], 'exam_score': [100, 90, 95, 80], 'project_score': [95, 92, 90, 85]})

# --- Семестр 2 ---
grades_math_s2 = pd.DataFrame({'student_id': ["S01", "S02", "S05", "S06"], 'exam_score': [88, 90, 75, 82], 'project_score': [92, 85, 80, 80]})
grades_physics_s2 = pd.DataFrame({'student_id': ["S01", "S02", "S05", "S06"], 'exam_score': [90, 82, 78, 88], 'project_score': [95, 80, 82, 85]})
grades_cs_s2 = pd.DataFrame({'student_id': ["S01", "S02", "S05", "S06"], 'exam_score': [98, 92, 80, 85], 'project_score': [90, 90, 85, 88]})

# --- Стипендії ---
scholarships = pd.DataFrame({
  'student_id': ["S01", "S03", "S05"],
  'scholarship_type': ["academic", "academic", "social"],
  'amount': [2000, 2000, 1500]
})

# --- 3. ЗАПИС ФАЙЛІВ ---

# Запис Excel-файлів
# Використовуємо pd.ExcelWriter, щоб додати кілька аркушів в один файл
try:
    with pd.ExcelWriter('data/semester1.xlsx', engine='openpyxl') as writer:
        students.to_excel(writer, sheet_name='students', index=False)
        grades_math_s1.to_excel(writer, sheet_name='grades_math', index=False)
        grades_physics_s1.to_excel(writer, sheet_name='grades_physics', index=False)
        grades_cs_s1.to_excel(writer, sheet_name='grades_cs', index=False)
    
    with pd.ExcelWriter('data/semester2.xlsx', engine='openpyxl') as writer:
        grades_math_s2.to_excel(writer, sheet_name='grades_math', index=False)
        grades_physics_s2.to_excel(writer, sheet_name='grades_physics', index=False)
        grades_cs_s2.to_excel(writer, sheet_name='grades_cs', index=False)
    
    # Запис CSV-файлу
    scholarships.to_csv("data/scholarships.csv", index=False)
    
    print("\n--- Python-скрипт успішно створив файли: ---")
    print(os.listdir(data_dir))

except Exception as e:
    print(f"\n--- !!! Виникла помилка під час запису файлів: {e} ---")
    print("Переконайтеся, що у вас встановлено 'openpyxl': !pip install openpyxl")

Папка data вже існує.

--- Python-скрипт успішно створив файли: ---
['scholarships.csv', 'semester1.xlsx', 'semester2.xlsx']


In [13]:
# Виконав(ла): Сидоренко Валерій | Група: (ваша група) | Викладач: Сидоренко В.М.

# --- 0. ЗАВАНТАЖЕННЯ БІБЛІОТЕК ---
import pandas as pd
import numpy as np

# Переконайтеся, що ви їх встановили:
# !pip install pandas openpyxl

# --- 1. ЗАВАНТАЖЕННЯ ДАНИХ ---

# 1.1. Завантажуємо довідник студентів (з S1) та стипендії
try:
    students_lookup = pd.read_excel("data/semester1.xlsx", sheet_name="students")
    scholarships_lookup = pd.read_csv("data/scholarships.csv")
    
    print("--- 1. Довідники завантажено ---")
    print(students_lookup.head())
    print("\n", scholarships_lookup.head())

except FileNotFoundError:
    print("!!! ПОМИЛКА: Файли з даними не знайдено. Переконайтеся, що Скрипт №1 відпрацював.")
    # Зупиняємо виконання, якщо файлів немає
    raise

# --- 2. ВЕРТИКАЛЬНЕ ОБ’ЄДНАННЯ (ETL для оцінок) ---

# 2.1. Функція для завантаження та обробки аркушів з оцінками
def load_grades(filepath, semester_name):
    # Завантажуємо аркуші
    grades_math = pd.read_excel(filepath, sheet_name="grades_math")
    grades_math['subject'] = 'math'
    
    grades_physics = pd.read_excel(filepath, sheet_name="grades_physics")
    grades_physics['subject'] = 'physics'
    
    grades_cs = pd.read_excel(filepath, sheet_name="grades_cs")
    grades_cs['subject'] = 'cs'
    
    # Об'єднуємо їх вертикально
    semester_grades = pd.concat([grades_math, grades_physics, grades_cs], ignore_index=True)
    semester_grades['semester'] = semester_name
    return semester_grades

# 2.2. Створюємо grades_s1 та grades_s2
grades_s1 = load_grades("data/semester1.xlsx", "S1")
grades_s2 = load_grades("data/semester2.xlsx", "S2")

# 2.3. Об'єднуємо S1 і S2 у all_grades
all_grades = pd.concat([grades_s1, grades_s2], ignore_index=True)

print("\n\n--- 2. Вертикальне об'єднання (all_grades) ---")
print(all_grades.head()) # Перші рядки (S1)
print("\n", all_grades.tail()) # Останні рядки (S2)


# --- 3. ГОРИЗОНТАЛЬНЕ ОБ’ЄДНАННЯ ---

# Створюємо єдиний аналітичний датафрейм (ADF)
# Додаємо інформацію про студентів (left join)
analytical_df = pd.merge(
    all_grades,
    students_lookup,
    on="student_id",
    how="left"
)

# Додаємо інформацію про стипендії (left join)
analytical_df = pd.merge(
    analytical_df,
    scholarships_lookup,
    on="student_id",
    how="left"
)

print("\n\n--- 3. Горизонтальне об'єднання (analytical_df) ---")
# `amount` буде NaN (Not a Number) для тих, хто не має стипендії
print(analytical_df)


# --- 4. АГРЕГАЦІЯ ТА АНАЛІЗ ---

# 4.1. Розрахунок загального балу
analytical_df['total_score'] = 0.6 * analytical_df['exam_score'] + 0.4 * analytical_df['project_score']

# 4.2. Додаємо допоміжні стовпці для агрегації
# (True/False перетворюються на 1/0 при .sum() та .mean())
analytical_df['is_high_achiever'] = analytical_df['total_score'] >= 90
analytical_df['has_scholarship'] = analytical_df['scholarship_type'].notna() # True, якщо стипендія є

# 4.3. Групування та агрегація
analysis_by_group_semester = analytical_df.groupby(['group', 'semester']).agg(
    # 'total_score': 'mean' - старий синтаксис, 'col_name': ('col_name', 'agg_func') - новий
    mean_total_score=('total_score', 'mean'),
    high_achievers_count=('is_high_achiever', 'sum'),
    scholarship_share=('has_scholarship', 'mean')
).reset_index() # .reset_index() повертає 'group' і 'semester' у стовпці

print("\n\n--- 4. Агрегація та Аналіз ---")
print(analysis_by_group_semester)


# --- 5. ФІЛЬТРУЮЧЕ З’ЄДНАННЯ (Емуляція semi_join та anti_join) ---

# 5.1. Визначення student_id, які отримували стипендію в обох семестрах
# Логіка: (Студенти S1 зі стипендією) semi_join (Студенти S2 зі стипендією)

# Студенти S1 зі стипендією (внутрішнє об'єднання S1 та стипендій)
s1_schol_students = pd.merge(
    grades_s1, scholarships_lookup, on='student_id', how='inner'
)['student_id'].unique()

# Студенти S2 зі стипендією (внутрішнє об'єднання S2 та стипендій)
s2_schol_students = pd.merge(
    grades_s2, scholarships_lookup, on='student_id', how='inner'
)['student_id'].unique()

# Емуляція semi_join: беремо ID з s1_schol_students, які є в s2_schol_students
students_in_both_mask = pd.Series(s1_schol_students).isin(s2_schol_students)
students_in_both = pd.DataFrame({
    'student_id': s1_schol_students[students_in_both_mask]
})

print("\n\n--- 5.1. Студенти зі стипендією в обох семестрах (semi_join) ---")
# Повинен бути 'S01' (за нашими даними)
print(students_in_both)


# 5.2. Визначення студентів, які втратили стипендію після S1
# Логіка: (Студенти S1 зі стипендією) anti_join (Усі студенти S2)

# Усі унікальні студенти S2
s2_all_students_ids = grades_s2['student_id'].unique()

# Емуляція anti_join: беремо ID з s1_schol_students, яких НЕМАЄ в s2_all_students_ids
lost_students_mask = ~pd.Series(s1_schol_students).isin(s2_all_students_ids)
lost_scholarship_students = pd.DataFrame({
    'student_id': s1_schol_students[lost_students_mask]
})

print("\n\n--- 5.2. Студенти, що втратили стипендію (S1+schol, anti_join S2) ---")
# Повинен бути 'S03' (за нашими даними)
print(lost_scholarship_students)

--- 1. Довідники завантажено ---
  student_id          full_name  group faculty
0        S01     Іваненко Петро  IT-21    FICT
1        S02    Коваленко Марія  IT-21    FICT
2        S03  Сидоренко Валерій  IT-22    FICT
3        S04      Мельник Олена  IT-22    FICT
4        S05     Кравчук Андрій  IT-21    FICT

   student_id scholarship_type  amount
0        S01         academic    2000
1        S03         academic    2000
2        S05           social    1500


--- 2. Вертикальне об'єднання (all_grades) ---
  student_id  exam_score  project_score  subject semester
0        S01          85             90     math       S1
1        S02          92             88     math       S1
2        S03          77             80     math       S1
3        S04          60             65     math       S1
4        S01          95             90  physics       S1

    student_id  exam_score  project_score  subject semester
19        S06          88             85  physics       S2
20        S01 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a083947-94ba-475d-8730-27cff0574f54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>